In [7]:
import pandas as pd
import boto3
from io import StringIO, BytesIO
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

# Adapter Layer

In [8]:
def read_csv_to_df(bucket, key, decoding = 'utf-8', sep=','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index = False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def return_objects(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    max_date =  datetime.strptime(arg_date, src_format).date() + timedelta(days=7) # I get the data of one week (7 days)
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date and datetime.strptime(obj.key.split('/')[0], src_format).date() < max_date] #filter objects of the bucket
    #print(objects)
    return objects   

# Application Layer

In [16]:
def extract(bucket, objects):
    df = pd.concat([read_csv_to_df(bucket, obj.key) for obj in objects], ignore_index = True)
    return df
    
def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace = True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['EndPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index = False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price']*100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals = 2)
    df = df[df.Date >= arg_date] # we filter from the arg_date forward
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%m%S") + trg_format
    write_df_to_s3(bucket, df, key)

def etl_report1(src_bucket, trg_bucket, objects, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, objects)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

# Main function entrypoints

In [18]:
def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2022-01-04'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'etl-pipelines-kspyrikos'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'report_'
    trg_format = '.parquet'

    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
   
    # Run Application
    objects = return_objects(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, objects, columns, arg_date, trg_key, trg_format)

# Run


In [19]:
main()

# Read the uploaded file

In [20]:
trg_bucket = 'etl-pipelines-kspyrikos'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
   
for obj in bucket_trg.objects.all():
    print(obj.key)

report_20231118_191119.parquet
report_20231118_221156.parquet


In [27]:
obj.key

'report_20231118_221156.parquet'

In [28]:
prq_obj = bucket_trg.Object(key=obj.key).get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data) # I can use 'read_parquet', because I used 'BytesIO'
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
1,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
2,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
3,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
4,AT000000STR1,2022-01-10,37.40,36.60,36.50,37.40,64,-2.01
...,...,...,...,...,...,...,...,...
15949,XS2376095068,2022-01-04,41.19,41.36,41.19,41.36,0,0.77
15950,XS2376095068,2022-01-05,40.99,41.02,40.77,41.02,250,-0.81
15951,XS2376095068,2022-01-06,38.15,37.85,37.85,38.15,7,-7.73
15952,XS2376095068,2022-01-07,36.86,36.71,36.71,37.41,2541,-3.02


# Functional solution

In [ ]:
arg_date = '2022-01-04'
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

s3 = boto3.resource('s3')

bucket = s3.Bucket('xetra-1234')

bucket_obj1 = bucket.objects.filter(Prefix='dataset/2021-03-15/')
bucket_obj2 = bucket.objects.filter(Prefix='dataset/2021-03-16/')

#desired_dates = ['2022-01-03', '2022-01-04']


In [ ]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt] #filter objects of the bucket
objects

In [ ]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')
bucket

# Read multiple files

In [ ]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in bucket.objects.all():
    #for date in desired_dates:
    if date in obj.key:
        csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
        data = StringIO(csv_obj)
        df = pd.read_csv(data, delimiter=',')
        df_all = pd.concat([df_all, df], ignore_index=True, sort=False).dropna(axis=1, how='all')
df_all

In [ ]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]
df_all

In [ ]:
df_all.dropna(inplace=True)
df_all

# Transformations

## Get opening price per ISIN and day

In [ ]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
df_all

In [ ]:
df_all[df_all['ISIN']=='DE000A0HN5C6']

## Get closing price per ISIN and day

In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['EndPrice'].transform('last')
df_all

## Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index = False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
df_all

## Percent Change Prev Closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
df_all

In [ ]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price']*100
df_all.drop(columns=['prev_closing_price'], inplace=True)
df_all = df_all.round(decimals = 2)
df_all = df_all[df_all.Date >= arg_date] # we filter from the arg_date forward
df_all

# Write to S3

In [ ]:
key = 'report_'+ datetime.today().strftime("%Y%m%d_%H%m%S")+'.parquet'
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index = False)
bucket_target = s3.Bucket('etl-pipelines-kspyrikos')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

## Read the uploaded file

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_target.Object(key='report_20231118_191119.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data) # I can use 'read_parquet', because I used 'BytesIO'
df_report